In [ ]:
# STEP 1 — Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# STEP 2 — Import Libraries
import pandas as pd
import numpy as np
import re, string, torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
# STEP 3 — Load Dataset
path = "/content/drive/MyDrive/major_project/IFND_dataset/IFND.csv"
df = pd.read_csv(path, encoding="latin1")
print("✅ Dataset loaded successfully!")
print("Shape:", df.shape)
df.head()

✅ Dataset loaded successfully!
Shape: (56714, 7)


,id,Statement,Image,Web,Category,Date,Label
0,2,"WHO praises India's Aarogya Setu app, says it ...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
1,3,"In Delhi, Deputy US Secretary of State Stephen...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,VIOLENCE,Oct-20,TRUE
2,4,LAC tensions: China's strategy behind delibera...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,TERROR,Oct-20,TRUE
3,5,India has signed 250 documents on Space cooper...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
4,6,Tamil Nadu chief minister's mother passes away...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,ELECTION,Oct-20,TRUE


In [ ]:
# =========================
# Cell 4.1 — Init & Unicode/Control cleanup
# =========================
import unicodedata, re, string

df = df.copy()
df['Statement'] = df['Statement'].astype(str)
df['text_clean'] = df['Statement']

def normalize_unicode(s: str) -> str:
    s = unicodedata.normalize('NFKC', s)
    s = ''.join(ch for ch in s if (ch.isprintable() or ch in ['\n', '\t', ' ']))
    s = re.sub(r'[\u200B-\u200D\uFEFF]', '', s)  # zero-width
    return s

df['text_clean'] = df['text_clean'].map(normalize_unicode)
print("Done 4.1 → unicode/control cleaned. Shape:", df.shape)


Done 4.1 → unicode/control cleaned. Shape: (56714, 8)


In [ ]:
# =========================
# Cell 4.1a — Mojibake Fix
# =========================
try:
    import ftfy
    def fix_mojibake(s: str) -> str:
        return ftfy.fix_text(str(s))
except Exception:
    def fix_mojibake(s: str) -> str:
        return str(s)

MOJIBAKE_MAP = {
    "": "'", "": "'",
    "Â": "", "Ã": "", "Ô": "", "Û": "", "Ò": "",
    "Û¢": "₹",
    "â€”": "—", "â€“": "–", "â€¦": "…",
    "â€˜": "'", "â€™": "'", "â€œ": '"', "â€\x9d": '"',
    "Ã—": "×",
}

def manual_unmojify(s: str) -> str:
    for bad, good in MOJIBAKE_MAP.items():
        if bad in s:
            s = s.replace(bad, good)
    s = ''.join(ch for ch in s if (ch.isprintable() or ch in ['\n','\t',' ']))
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def clean_mojibake(s: str) -> str:
    return manual_unmojify(fix_mojibake(s))

df['Statement']  = df['Statement'].map(clean_mojibake)
df['text_clean'] = df['text_clean'].map(clean_mojibake)
print("Done 4.1a → mojibake fixed.")


Done 4.1a → mojibake fixed.


In [ ]:
# =========================
# Cell 4.1b — Residual apostrophe fixes
# =========================
EXTRA_APOS_MAP = {"Õ": "'", "º": "'", "ª": "'", "´": "'", "`": "'", "’": "'", "‘": "'"}

def fix_extra_apostrophes(s: str) -> str:
    for bad, good in EXTRA_APOS_MAP.items():
        if bad in s:
            s = s.replace(bad, good)
    s = re.sub(rf"\b([A-Za-z]{{2,}})[Õºª´`]s\b", r"\1's", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["Statement"]  = df["Statement"].map(fix_extra_apostrophes)
df["text_clean"] = df["text_clean"].map(fix_extra_apostrophes)
print("Done 4.1b → residual apostrophes fixed.")


Done 4.1b → residual apostrophes fixed.


In [ ]:
# =========================
# ✅ Updated Cell 4.1c — Safe Symbol & Punctuation Cleanup
# (Does NOT break: 1.5 , 10:30 pm , 80:20 , 5,234,235 type patterns)
# =========================
import re

RESIDUAL_MAP = {
    "Ð": "—", "ð": "—",
    "Ê": "", "Ë": "",
    "�": "", "™": "", "©": "", "®": "",
    "•": " ", "·": " ",
}

# Regex protection patterns (things we should NOT modify)
decimal_pattern = re.compile(r'\d+\.\d+')          # e.g. 1.5, 2.75
time_pattern    = re.compile(r'\b\d{1,2}:\d{2}(?:\s?(?:AM|PM|am|pm))?\b')  # 10:30, 09:45 pm
ratio_pattern   = re.compile(r'\b\d+:\d+\b')       # 80:20, 3:2
comma_number    = re.compile(r'\d{1,3}(?:,\d{2,3})+')  # 5,234,235 or 12,34,567

def fix_residuals_safe(s: str) -> str:
    s_original = s = str(s)

    # 🔹 STEP 1: Temporarily protect critical patterns
    placeholders = {}
    for pat_name, pattern in {
        "DEC": decimal_pattern,
        "TIME": time_pattern,
        "RATIO": ratio_pattern,
        "COMMA": comma_number
    }.items():
        for match in pattern.findall(s):
            key = f"<{pat_name}_{len(placeholders)}>"
            placeholders[key] = match
            s = s.replace(match, key)

    # 🔹 STEP 2: Replace bad unicode/symbols safely
    for bad, good in RESIDUAL_MAP.items():
        s = s.replace(bad, good)

    # 🔹 STEP 3: Clean stray bullets or weird start chars
    s = re.sub(r'^\s*[?•·]\s*', '', s)

    # 🔹 STEP 4: Fix em-dash spacing → word — word
    s = re.sub(r'\s*—\s*', ' — ', s)

    # 🔹 STEP 5: Remove extra space before punctuation (except colon or decimals)
    s = re.sub(r'\s+([,!?:;])', r'\1', s)

    # 🔹 STEP 6: Add space after punctuation (, ! ? ;) except numbers/time
    s = re.sub(r'(?<=[,!?:;])(?=[^\s])', ' ', s)

    # 🔹 STEP 7: Collapse multiple spaces
    s = re.sub(r'\s{2,}', ' ', s).strip()

    # 🔹 STEP 8: Restore protected parts
    for key, value in placeholders.items():
        s = s.replace(key, value)

    return s

df['Statement']  = df['Statement'].map(fix_residuals_safe)
df['text_clean'] = df['text_clean'].map(fix_residuals_safe)

print("✅ Updated 4.1c applied safely — no more breaking decimals, times, or numbers.")


✅ Updated 4.1c applied safely — no more breaking decimals, times, or numbers.


In [ ]:
# =========================
# Cell 4.1d — Fix stray Ó and dangling quotes
# =========================
import re

def fix_O_and_quotes(s: str) -> str:
    s = str(s)

    # 1) Ó between spaces -> em dash
    # e.g., "protest Ó SC" -> "protest — SC"
    s = re.sub(r'\s*Ó\s*', ' — ', s)

    # 2) Rare possessive mojibake: wordÓs -> word's
    s = re.sub(r"\b([A-Za-z]{2,})Ós\b", r"\1's", s)

    return s

# apply to both views so they stay in sync
df["Statement"]  = df["Statement"].map(fix_O_and_quotes)
df["text_clean"] = df["text_clean"].map(fix_O_and_quotes)

print("4.1d → fixed mid-sentence Ó (→ —) and removed dangling quotes.")


4.1d → fixed mid-sentence Ó (→ —) and removed dangling quotes.


In [ ]:
# =========================
# Cell 4.1e — Deep Mojibake & Residual Encoding Fix
# (Fixes: Ã¢, Ã¯Â¿Â½, Ã Â¤Â etc.)
# =========================
!pip install ftfy
import ftfy, codecs

def deep_mojibake_fix(s: str) -> str:
    if not isinstance(s, str):
        return str(s)
    text = s
    # Recursive multi-pass decode attempts
    for _ in range(3):
        try:
            text = ftfy.fix_text(text)
            text = codecs.decode(text.encode("latin1", "ignore"), "utf-8", "ignore")
        except Exception:
            break
    # Clean up any remaining mojibake patterns
    text = re.sub(r'[Ã¢Âï¤]+', '', text)
    text = re.sub(r'\s{2,}', ' ', text).strip()
    return text

df['Statement']  = df['Statement'].map(deep_mojibake_fix)
df['text_clean'] = df['text_clean'].map(deep_mojibake_fix)
print("✅ Done 4.1e → Deep mojibake fixed (Ã¢, Ã¯Â¿Â½, Ã Â¤Â etc). Shape:", df.shape)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
✅ Done 4.1e → Deep mojibake fixed (Ã¢, Ã¯Â¿Â½, Ã Â¤Â etc). Shape: (56714, 8)


In [ ]:
# =========================
# Cell 4.2 — Drop Null/Empty Statements & Labels
# =========================
df['text_clean'] = df['text_clean'].str.strip()
if 'Label' in df.columns:
    df['Label'] = df['Label'].astype(str).str.strip()

before = len(df)
df = df[ df['text_clean'].notna() & (df['text_clean'].str.len() > 0) ]
df = df[ df['Label'].notna() & (df['Label'].str.len() > 0) ]
after = len(df)
print(f"Done 4.2 → dropped empties. Removed {before-after}, new shape: {df.shape}")


Done 4.2 → dropped empties. Removed 0, new shape: (56714, 8)


In [ ]:
# =========================
# Cell 4.3 — Remove Exact Duplicates
# =========================
before = len(df)
df = df.drop_duplicates(subset=['text_clean'])
after = len(df)
print(f"Done 4.3 → exact duplicates removed: {before-after}. Shape: {df.shape}")


Done 4.3 → exact duplicates removed: 929. Shape: (55785, 8)


In [ ]:
# =========================
# Cell 4.4 — Remove 'Normalized' Near-Duplicates
# =========================
def normalized_signature(s: str) -> str:
    s = s.lower()
    s = s.translate(str.maketrans('', '', string.punctuation))
    s = re.sub(r'\s+', ' ', s).strip()
    return s

sig_col = '__sig__'
df[sig_col] = df['text_clean'].map(normalized_signature)
before = len(df)
df = df.drop_duplicates(subset=[sig_col]).drop(columns=[sig_col])
after = len(df)
print(f"Done 4.4 → normalized near-dupes removed: {before-after}. Shape: {df.shape}")
print("Shape:", df.shape)


Done 4.4 → normalized near-dupes removed: 3026. Shape: (52759, 8)
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.5 — Label Normalization
# =========================
label_map = {
    'TRUE':'TRUE','T':'TRUE','REAL':'TRUE','1':'TRUE',
    'FALSE':'FALSE','F':'FALSE','FAKE':'FALSE','0':'FALSE',
    'OTHER':'OTHER','MIXED':'OTHER','HALF TRUE':'OTHER','PARTLY TRUE':'OTHER','PARTLY':'OTHER'
}
def canon_label(x: str) -> str:
    return label_map.get(x.upper().strip(), x.upper().strip())

df['Label'] = df['Label'].map(canon_label)
allowed = {'TRUE','FALSE','OTHER'}
before = len(df)
df = df[df['Label'].isin(allowed)]
after = len(df)
print(f"Done 4.5 → labels normalized. Dropped {before-after}. Shape: {df.shape}")
print(df['Label'].value_counts())
print("Shape:", df.shape)


Done 4.5 → labels normalized. Dropped 0. Shape: (52759, 8)
Label
TRUE     36454
FALSE    16305
Name: count, dtype: int64
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.6 — Whitespace & Quotes Normalization
# =========================
def normalize_spaces_quotes(s: str) -> str:
    s = s.replace('“','"').replace('”','"').replace('‘',"'").replace('’',"'")
    s = re.sub(r'[ \t]+', ' ', s)
    s = re.sub(r'\s*\n\s*', ' ', s)
    return s.strip()

df['text_clean'] = df['text_clean'].map(normalize_spaces_quotes)
print("Done 4.6 → whitespace/quotes normalized.")
print("Shape:", df.shape)


Done 4.6 → whitespace/quotes normalized.
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.7 — Remove HTML & Inline Credits
# =========================
html_tag = re.compile(r'<[^>]+>')
credit_patterns = [
    r'(?:image\s*source\s*:\s*.*)$',
    r'(?:photo\s*:\s*.*)$',
    r'(?:via\s*[:\-]\s*.*)$'
]
def strip_html_boiler(s: str) -> str:
    s = html_tag.sub(' ', s)
    for cp in credit_patterns:
        s = re.sub(cp, '', s, flags=re.IGNORECASE)
    return re.sub(r'\s+', ' ', s).strip()

df['text_clean'] = df['text_clean'].map(strip_html_boiler)
print("Done 4.7 → HTML/boilerplate removed.")
print("Shape:", df.shape)

Done 4.7 → HTML/boilerplate removed.
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.8 — Replace URLs/Emails/Handles/Hashtags
# =========================
url_re = re.compile(r'\b((?:https?://|www\.)\S+)\b', flags=re.IGNORECASE)
email_re = re.compile(r'\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b', flags=re.IGNORECASE)
mention_re = re.compile(r'(?<!\w)@\w+')
hashtag_re = re.compile(r'(?<!\w)#\w+')

def replace_specials(s: str) -> str:
    s = url_re.sub('<URL>', s)
    s = email_re.sub('<EMAIL>', s)
    s = mention_re.sub('<USER>', s)
    s = hashtag_re.sub('<HASHTAG>', s)
    return s

df['text_clean'] = df['text_clean'].map(replace_specials)
print("Done 4.8 → URLs/emails/handles/hashtags tokenized.")
print("Shape:", df.shape)


Done 4.8 → URLs/emails/handles/hashtags tokenized.
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.9 — Mild Symbol Cleaning
# =========================
keep = set(list(".,!?:;'\"()- %₹$€£"))
def clean_symbols(s: str) -> str:
    out = []
    for ch in s:
        if ch.isalnum() or ch in keep:
            out.append(ch)
        else:
            pass
    s = ''.join(out)
    return re.sub(r'\s+', ' ', s).strip()

df['text_clean'] = df['text_clean'].map(clean_symbols)
print("Done 4.9 → mild symbol cleaning.")
print("Shape:", df.shape)


Done 4.9 → mild symbol cleaning.
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.10 — Robust Number Normalization (Indian + Western + spaced commas)
# =========================
REPLACE_THRESHOLD = 5  # 5+ digits => <NUM>

# Allow optional spaces after comma groups
west  = r'\d{1,3}(?:,\s?\d{3})+(?:\.\d+)?'        # 1,234,567(.xx) or 1, 234, 567(.xx)
ind   = r'\d{1,2}(?:,\s?\d{2})+,\s?\d{3}(?:\.\d+)?' # 12,34,567(.xx) or 12, 34, 567(.xx)
plain = r'\d{5,}(?:\.\d+)?'                         # 12345(.xx)

_num_pat = re.compile(
    rf'(?P<prefix>(?<!\w)[₹$€£]?)'
    rf'(?P<num>({ind}|{west}|{plain}))'
    rf'(?P<suffix>%?)'
)
_year_pat = re.compile(r'^(19|20)\d{2}$')

def _should_replace(num_str: str, suffix: str) -> bool:
    if suffix == '%':  # keep percentages
        return False
    # remove commas + spaces to count true digits
    digits_only = re.sub(r'[^\d]', '', num_str)
    if _year_pat.fullmatch(digits_only or ''):
        return False
    return len(digits_only) >= REPLACE_THRESHOLD

def normalize_numbers(s: str) -> str:
    # 1) collapse spaces after commas inside numeric groups: "59, 88, 822" -> "59,88,822"
    s = re.sub(r'(?<=\d),\s+(?=\d)', ',', s)

    # 2) now replace long numbers with <NUM>, keep years & percentages
    def repl(m):
        prefix = m.group('prefix') or ''
        num    = m.group('num')
        suffix = m.group('suffix') or ''
        return f"{prefix}<NUM>{suffix}" if _should_replace(num, suffix) else f"{prefix}{num}{suffix}"

    return _num_pat.sub(repl, s)

df['text_clean'] = df['text_clean'].map(normalize_numbers)
print("Done 4.10 → number normalization fixed (commas/years/% handled).")
print("Shape:", df.shape)


Done 4.10 → number normalization fixed (commas/years/% handled).
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.11 — Case Handling (No Lowercasing)
# =========================
print("Done 4.11 → case preserved.")
print("Shape:", df.shape)


Done 4.11 → case preserved.
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.11b — Possessive repair from raw Statement
# =========================
def repair_possessives(row):
    raw = str(row['Statement'])
    clean = str(row['text_clean'])
    for m in re.finditer(r"\b([A-Za-z]{2,})'s\b", raw):
        w = m.group(1)
        clean = re.sub(rf"\b{re.escape(w)}as\b", rf"{w}'s", clean)
    return clean

df['text_clean'] = df.apply(repair_possessives, axis=1)
print("Done 4.11b → possessive glitches repaired.")
print("Shape:", df.shape)


Done 4.11b → possessive glitches repaired.
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.11c — Quote repair (avoid ' → a)  [UPDATED]
# Must run AFTER 4.11b
# =========================
def repair_quote_glitches(row):
    raw   = str(row['Statement'])
    clean = str(row['text_clean'])

    # Case A: raw has word' (trailing apostrophe) → clean wrongly has worda
    # Handle when 'a' is followed by space/END or punctuation like , . ; : ! ?
    for m in re.finditer(r"\b([A-Za-z]{2,})['’](?=\s|$|[,.;:!?])", raw):
        w = m.group(1)
        clean = re.sub(rf"\b{re.escape(w)}a(?=\s|$|[,.;:!?])", rf"{w}'", clean)

    # Case B: raw has 'word' (quoted word) → clean wrongly has worda
    for m in re.finditer(r"['’]([A-Za-z]{2,})['’]", raw):
        w = m.group(1)
        clean = re.sub(rf"\b{re.escape(w)}a(?=\s|$|[,.;:!?])", rf"{w}'", clean)

    return clean

df['text_clean'] = df.apply(repair_quote_glitches, axis=1)
print("4.11c → quote glitches fixed incl. punctuation (no more mentalitya, / Squada etc.).")


4.11c → quote glitches fixed incl. punctuation (no more mentalitya, / Squada etc.).


In [ ]:
# =========================
# Cell 4.12 — No Lemmatization/Stopword Removal (confirm)
# =========================
assert 'text_clean' in df.columns and df['text_clean'].notna().all()
print("Done 4.12 → confirmed: no lemma/stopword removal.")
print("Shape:", df.shape)

Done 4.12 → confirmed: no lemma/stopword removal.
Shape: (52759, 8)


In [ ]:
# =========================
# Cell 4.13 — Source-Boilerplate Mitigation (light)
# =========================
pubs = []
if 'Web' in df.columns:
    pubs = sorted(df['Web'].dropna().astype(str).str.strip().unique().tolist())

extra_variants = ['DNAINDIA', 'DNA INDIA', 'DNA-INDIA']
pub_tokens = list({p.upper() for p in pubs} | set(extra_variants))
pub_tokens = [re.escape(p) for p in pub_tokens if p and p != 'NAN']
pub_tail_re = re.compile(rf'\s*[-–—]?\s*(?:{"|".join(pub_tokens)})\s*$', flags=re.IGNORECASE) if pub_tokens else None

def strip_publisher_tail(s: str) -> str:
    if pub_tail_re:
        s = pub_tail_re.sub('', s)
        s = re.sub(r'\s+', ' ', s).strip()
    return s

df['text_clean'] = df['text_clean'].map(strip_publisher_tail)
print("Done 4.13 → light publisher-tail stripping.")
print("Shape:", df.shape)


Done 4.13 → light publisher-tail stripping.
Shape: (52759, 8)


In [ ]:
# =========================
# 4.14 Token Limit + Minimum Length + Save
# =========================

import re
from transformers import RobertaTokenizerFast

# 1) Load tokenizer for token length checking
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# 2) Function to compute token length
def get_token_len(text):
    return len(tokenizer.encode(text, add_special_tokens=True, truncation=False))

# 3) Remove rows where token length > 512 (RoBERTa/DeBERTa limit)
df['__tok_len__'] = df['text_clean'].apply(get_token_len)
before = len(df)
df = df[df['__tok_len__'] <= 512]
after = len(df)
print(f"✔ Removed {before - after} rows with token length > 512")
df = df.drop(columns=['__tok_len__'])

# 4) Minimum length filter (keep only text with ≥3 meaningful tokens)
def token_count(s: str) -> int:
    return len([t for t in re.findall(r"\w+|[.,!?:;'\"()-]", s)])

before = len(df)
df['__len__'] = df['text_clean'].map(token_count)
df = df[df['__len__'] >= 3].drop(columns=['__len__'])
after = len(df)
print(f"✔ Removed {before - after} rows with token length < 3")

# 5) Reorder columns properly
cols = df.columns.tolist()
order = ['id','Statement','text_clean','Label','Category','Web','Date','Image']
final_cols = [c for c in order if c in cols] + [c for c in cols if c not in order]
df = df[final_cols]

# 6) Save CSV
print("✅ Final dataset shape:", df.shape)
print(df[['Statement','text_clean','Label']].head(3))

out_path = "/content/drive/MyDrive/major_project/IFND_dataset/IFND_clean3.csv"
df.to_csv(out_path, index=False, encoding='utf-8')
print("✅ Saved cleaned dataset to:", out_path)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

✔ Removed 0 rows with token length > 512
✔ Removed 5 rows with token length < 3
✅ Final dataset shape: (52754, 8)
                                           Statement  \
0  WHO praises India's Aarogya Setu app, says it ...   
1  In Delhi, Deputy US Secretary of State Stephen...   
2  LAC tensions: China's strategy behind delibera...   

                                          text_clean Label  
0  WHO praises India's Aarogya Setu app, says it ...  TRUE  
1  In Delhi, Deputy US Secretary of State Stephen...  TRUE  
2  LAC tensions: China's strategy behind delibera...  TRUE  
✅ Saved cleaned dataset to: /content/drive/MyDrive/major_project/IFND_dataset/IFND_clean3.csv
